In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model="llama-3.1-70b-versatile",
    groq_api_key = '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'
    
    # other params...
)
response = llm.invoke("the first person discovered satellite.....")
print(response.content)

The first person to discover a satellite was Galileo Galilei.  On January 7, 1610, Galileo discovered four moons (satellites) orbiting Jupiter: Io, Europa, Ganymede, and Callisto. These four moons are now known as the Galilean moons in his honor.


In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.mercedes-benz.com/enUS/machine-learning-engineer-ai-products-140531-MER0003CK2")
page_data = loader.load().pop().page_content
print(page_data)





Machine Learning Engineer (AI Products) | Mercedes-Benz > Career > Job Search > Job Postings





































































 DE  EN Provider/PrivacyCompanyInnovationResponsibilityCareersInvestorsPressProductsCareersJob searchMachine Learning Engineer (AI Products)Job category:Forschung & Entwicklung incl. DesignDepartment:RD ChinaOrganization:Mercedes-Benz Group China Ltd.Location:Mercedes-Benz Group China Ltd., BeijingStart date:immediatelyPublication date:13.10.2024Job number:MER0003CK2Working time:Vollzeit ApplyTasksAs a world leading OEM, Mercedes-benz is looking for the experts who share the passion and enthusiasm to develop AI products in our MBOS. Specifically we are looking for self-motive, knowledgeable machine learning engineers who have a strong background in machine learning, large language model and their applications in automotive industry to join our team.
Responsibilities:•    Design, develop, and maintain AI based automotive ap

In [6]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:

    {page_data}

    ### INSTRUCTION:

    The scraped text is from the career's page of a website.

    Your job is to extract the job postings and return them in JSON format containin following keys: role,xperience, skills and description. Only return the valid JSON.
    
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)
type(res.content)


```
{
  "role": "Machine Learning Engineer (AI Products)",
  "experience": "Proven experience as a Machine Learning Engineer or similar role",
  "skills": [
    "Master degree in Computer Science, Engineering, or a related subject",
    "Experience with machine learning frameworks (like PyTorch or Tensorflow) and libraries (like scikit-learn)",
    "Proficient in Python and familiarity with Golang, Java or C++",
    "Understanding of data structures and data modeling",
    "Extensive experience with machine learning, deep learning, and applications of large language models",
    "Experience in developing automotive applications is a plus",
    "Experience in deploying models on edge device is a plus",
    "Experience with cloud services (e.g., Azure, Tencent)"
  ],
  "description": "Design, develop, and maintain AI based automotive applications. Train, tune, and optimize machine learning and deep learning models to improve performance and efficiency. Build up LLM-based applications suc

str

In [7]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)
type(json_res)

{'role': 'Machine Learning Engineer (AI Products)', 'experience': 'Proven experience as a Machine Learning Engineer or similar role', 'skills': ['Master degree in Computer Science, Engineering, or a related subject', 'Experience with machine learning frameworks (like PyTorch or Tensorflow) and libraries (like scikit-learn)', 'Proficient in Python and familiarity with Golang, Java or C++', 'Understanding of data structures and data modeling', 'Extensive experience with machine learning, deep learning, and applications of large language models', 'Experience in developing automotive applications is a plus', 'Experience in deploying models on edge device is a plus', 'Experience with cloud services (e.g., Azure, Tencent)'], 'description': 'Design, develop, and maintain AI based automotive applications. Train, tune, and optimize machine learning and deep learning models to improve performance and efficiency. Build up LLM-based applications such as a RAG system within given context. Utilize t

dict

In [8]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [9]:
import uuid
import chromadb

client = chromadb.PersistentClient('VectorStore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents = row["Techstack"],
                    metadatas = {"links":row["Links"]},
                      ids =[str(uuid.uuid4())])
    

In [10]:
# Perform the query
results = collection.query(query_texts=["Experience in java"], n_results=3)

# Extract the metadata
links = results['metadatas']  # Use 'metadatas' instead of 'metadata'
links


[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}]]

In [11]:
job = json_res
job['skills']

['Master degree in Computer Science, Engineering, or a related subject',
 'Experience with machine learning frameworks (like PyTorch or Tensorflow) and libraries (like scikit-learn)',
 'Proficient in Python and familiarity with Golang, Java or C++',
 'Understanding of data structures and data modeling',
 'Extensive experience with machine learning, deep learning, and applications of large language models',
 'Experience in developing automotive applications is a plus',
 'Experience in deploying models on edge device is a plus',
 'Experience with cloud services (e.g., Azure, Tencent)']

In [12]:
results = collection.query(query_texts=job['skills'], n_results=3)

# Extract the metadata
links = results['metadatas']  # Use 'metadatas' instead of 'metadata'
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links

In [13]:
prompt_email = PromptTemplate.from_template(
     """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Seshu Swaraj, a JAVA development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
)
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Machine Learning Solutions for Automotive Applications

Dear Hiring Manager,

I came across the job description for a Machine Learning Engineer (AI Products) and was impressed by the exciting opportunity to design, develop, and maintain AI-based automotive applications. As a representative of AtliQ, an AI & Software Consulting company, I'd like to highlight our capabilities in fulfilling your needs.

At AtliQ, we have a proven track record of empowering numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in developing and deploying machine learning and deep learning models, including large language models.

We'd like to showcase some of our relevant portfolio pieces that demonstrate our expertise in machine learning and Python:

* https://example.com/ml-python-portfolio
* https://example.com/python-portfolio

Our team is well-versed in m